**Author**: stefan.m.janssen@gmail.com<br>
**Date**: 08 Aug 2016<br>
**language**: Python 3.5.2 :: Continuum Analytics, Inc.<br>
**conda enviroment**: micronota<br>
**license**: unlicensed<br>

# Filter EMP
Currently the EMP comprises ~34k samples. Sometimes we need to operate on smaller sample numbers. Thus, the merged mapping file contains 'subset_xxx' columns, where xxx indicates the number of samples in the subset. These columns are np.boolean and flags if a sample of the whole EMP is part of subset xxx.

There are several point where we need to filter full data-sets to those smaller subsets, e.g. BIOM tables. This notebook is an example of how to acomplish that. 

In [21]:
# a trick to make it transparent if executed on a barnacle node or on my machine with barnacles file system mounted to /media/barnacle
import socket
ROOT = "/media/barnacle/" if socket.gethostname() == 't440s' else "/"

In [22]:
%matplotlib inline

# make all necessary imports
import pandas as pd
import numpy as np
import sys
import seaborn as sn
import matplotlib.pyplot as plt
import os
import biom
from biom.util import biom_open

the next cell contains all parameters that might need to be changed

In [14]:
# file_mapping points to the merged EMP mapping file, which contains columns like 'subset_2000'.
file_mapping = ROOT + '/projects/emp/00-qiime-maps/merged/emp_qiime_mapping_latest.tsv'

# file_biom points to the full BIOM table
file_biom = ROOT + '/projects/emp/03-otus/01-closed-ref-greengenes/emp_cr_gg_13_8.biom'

In [23]:
#we load the mapping file into a Pandas dataframe
metadata = pd.read_csv(file_mapping, sep="\t", index_col=0, low_memory=False)

#we chose a (sub-)set of sample IDs which we want to have in our output set. E.g. all sampleIDs that have a np.True_ in the 'subset_2000' column, i.e. those 2000 samples that form the EMP 2000 sub-set 
idx = metadata[metadata.empo_3 == 'Animal distal gut'].index

In [20]:
filterBiomTable(idx, file_biom, "./filtered.biom")

Loading source biom table ... which holds 27398 samples.
Sub-sampling down to 4508 of which 894 are not in the input BIOM table.
Writing resulting biom table to './filtered.biom'
Generating summary.


In [19]:
# Reads a biom table from 'sourceBiomFile' and remains only those samples whose IDs are in the list 'sampleIDsToKeep'. Result will be written to the file 'targetBiomFile'

def filterBiomTable(sampleIDsToKeep, sourceBiomFile, targetBiomFile):
    print("Loading source biom table ...", end="")
    counts = biom.load_table(sourceBiomFile)
    
    #collect available sample IDs from input biom table (some IDs of the list 'sampleIDsToKeep' might not be in the input biom file, e.g. if we don't have sequence reads)
    counts_idx = counts.ids('sample')
    print(" which holds " + str(len(counts_idx)) + " samples.")

    toBeFiltered_idx = list(set(counts_idx) & set(sampleIDsToKeep))
    notFound = list(set(sampleIDsToKeep) - set(counts_idx))
    print("Sub-sampling down to " + str(len(toBeFiltered_idx)) + " of which " + str(len(notFound)) + " are not in the input BIOM table.")
    
    #actual filtering
    counts_subset = counts.filter(toBeFiltered_idx, axis='sample', inplace=False)
    
    #create the new filename
    print("Writing resulting biom table to '"+ targetBiomFile +"'")
    with biom_open(targetBiomFile, 'w') as f:
        counts_subset.to_hdf5(f, "EMP sub-set generation notebook")
        
    print("Generating summary.")
    inp = targetBiomFile
    out = targetBiomFile + ".summary.txt"
    !biom summarize-table -i "$inp" > "$out"